In [1]:
!pip install CoolProp

In [2]:
from CoolProp.CoolProp import PropsSI

In [3]:
!pip install iapws

In [4]:
from iapws import IAPWS97

Consider a heat exchanger operating under steady flow conditions. We assume that there is no pressure drop across the heat exchanger from the inlets to the outlets. Water is used to cool a refrigerant (R-134a). Here are the inlet (state 1) and outlet (state 2) conditions of the water:

In [6]:
T1_K = 15+273.15
p1_MPa = 300/1000

T2_K = 25+273.15
p2_MPa = p1_MPa

Here are the inlet (state 3) and outlet (state 4) properties of the refrigerant as well as its mass flow rate, which is given:

In [8]:
fluid = 'R134a'

T3_K = 70+273.15
p3_Pa = 1e6

T4_K = 35+273.15
p4_Pa = p3_Pa

mdot_r134a_kgPerMin = 6
mdot_r134a_kgPerS = mdot_r134a_kgPerMin/60

Determine:
- The mass flow rate of the cooling water.
- The heat transfer rate from the refrigerant to the water.

We can first analyze the water and refrigerant sections of the heat exchangers as separate control volumes:

Water: $\dot{Q}_\text{in} + \dot{m}_\text{water}h_1 = \dot{m}_\text{water}h_2$

Refrigerant: $\dot{m}_\text{R-134a}h_3 = \dot{Q}_\text{out} + \dot{m}_\text{R-134a}h_4$

Note that as heat is transferred from the refrigerant to the water, we have:

$\dot{Q}_\text{in} = \dot{Q}_\text{out}$

Therefore, we can combine the above equations and solve for the water mass flow rate:



$\dot{m}_\text{water} = \dot{m}_\text{R-134a} \frac{\left( h_3 - h_4 \right)}{\left( h_2 - h_1 \right)}$

We see from this that we need to know the enthalpies at the inlets and outlets of the heat exchanger. From that we can calculate the water mass flow rate. 

In [9]:
# calculate water properties
water_1 = IAPWS97(P=p1_MPa, T=T1_K)
water_2 = IAPWS97(P=p2_MPa, T=T2_K)
h1_kJperKg = water_1.h # kJ/kg
h2_kJperKg = water_2.h # kJ/kg

# convert to SI units
h1_JperKg = h1_kJperKg*1000; print(h1_JperKg) # J/kg
h2_JperKg = h2_kJperKg*1000; print(h2_JperKg) # J/kg

# calculate refrigerant enthalpy properties
h3_JperKg = PropsSI('H','P',p3_Pa,'T',T3_K,fluid); print(h3_JperKg) #J/kg
h4_JperKg = PropsSI('H','P',p4_Pa,'T',T4_K,fluid); print(h4_JperKg) #J/kg

63269.22110794279
105113.2796059467
452000.5965967065
248995.07776084734


In [10]:
mdot_water_kgPerS = mdot_r134a_kgPerS*(h3_JperKg-h4_JperKg)/(h2_JperKg-h1_JperKg)
print('(a): water mass flow rate: {0:1.4f} kg/s'.format(mdot_water_kgPerS))

(a): water mass flow rate: 0.4851 kg/s


Now we can determine the heat transfer rate from the refrigerant to the water from our formula above:

$\dot{m}_\text{R-134a}h_3 = \dot{Q}_\text{out} + \dot{m}_\text{R-134a}h_4$

Which we can rearrange as:

$\dot{Q}_\text{out} = \dot{m}_\text{R-134a} \left( h_3 - h_4 \right)$

In [11]:
Qout_r134a_JperS = mdot_r134a_kgPerS*(h3_JperKg - h4_JperKg)
print('(b): Q_out: {0:1.4f} J/s'.format(Qout_r134a_JperS))

(b): Q_out: 20300.5519 J/s


We would get the same answer if we solved for $Q_\text{in}$:

$\dot{Q}_\text{in} = \dot{m}_\text{water}\left( h_2 - h_1 \right)$

In [12]:
Qin_water_JperS = mdot_water_kgPerS*(h2_JperKg - h1_JperKg)
print('(b): Q_in: {0:1.4f} J/s'.format(Qin_water_JperS))

(b): Q_in: 20300.5519 J/s
